    # Clase - Computación Distribuida

    ## Pyspark Hands-on 
    #### Marcelo Medel Vergara - Diplomado Data Engineer USACH

## Instalación de librerías necesarias

Si utilizas Anaconda para administrar ambientes de desarrollo la mejor vía para asegurar que funcionen correctamente las librerías es instalando directo desde Conda. Adicionalmente se instalan todas las librerías que necesita Spark.
- `conda create -n pyspark-DE`
- `conda activate pyspark-DE`
- `conda install -c conda-forge pyspark python=3.10`

De igual forma se puede instalar a través de PIP, pero sin asegurar funcionamiento correcto ni la instalación de dependencias. Adicionalmente será necesario tener instalado Pandas.
- `conda create -n pyspark-DE python=3.10`
- `conda activate pyspark-DE`
- `pip install pyspark`
- `pip install pandas`.

Para Google Colab:
- `!pip install pyspark`

## SparkSession

SparkSession es una clase en PySpark que existe desde la versión 2.0 (2016) que simplifica la forma de trabajar con Spark, tanto en las configuraciones como en la manipulación de datos estructurados. 


#### Funcionalidades principales de SparkSession:

1. **Configura Spark**: Para profundizar en las configuraciones posibles de Spark, visite https://spark.apache.org/docs/latest/configuration.html.
    - `SparkSession.builder.appName("some name").**config("some config key,value")**.getOrCreate()`
    - `spark.conf.get("some config key")`

2. **Crear DataFrames**: permite leer y escribir (Input/Output) diversas fuentes de datos y crear DataFrames para la manipulación de datos.
    - `spark.createDataFrame(data [, schema])`
    - `spark.read.json("path to some json")`

3. **Ejecutar SQL**: facilita la ejecución de consultas en SQL sobre los DataFrames.
    - `spark.sql("query to some view")`
    
4. **Gestiona contexto de Spark**: facilita la configuración y acceso a diferentes componentes y funcionalidades de Spark
    - `spark.sql.shuffle.partitions`
    - `spark.executor.memory`
    - `spark.catalog.listTables()`
    - `spark.catalog.listColumns("someTable")`
    - `spark.udf.register("someName", someUdf)`

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("hands-on-pyspark")\
        .config("spark.executor.memory","2g")\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR") #ALL, OFF, ERROR, DEBUG, INFO, WARN
spark.active()

## DataFrame en Spark

Un DataFrame es una estructura de datos bidimensional similar a cualquier tabla en una base de datos estructurada. Algunas de las características que tiene un DataFrame en PySpark son:
- **Distribuido**: Los datos están distribuidos en un clúster de nodos, lo que permite el procesamiento paralelo.

- **Inmutable**: Cada transformación produce un nuevo DataFrame.

- **SQL**: Permite operaciones tipo SQL y ofrece una interfaz similar a Pandas pero a gran escala.

- **Conexiones diversas**: Puede leer datos de múltiples fuentes (ej: JSON, CSV, Parquet, JDBC).

- **Optimización Automática**: Utiliza *Catalyst Optimizer* para optimizar automáticamente las consultas.

### Creación de un DataFrame

In [25]:
from datetime import date
data = [
    {
        "nombre":"Marcelo",
        "nacimiento": date(1987,10,7),
        "mail":"marcelo@gmail.com",
        "x":5689,
        "y":15.26,
        "active":True
    },
    {
        "nombre":"Andrea",
        "nacimiento": date(1990,8,19),
        "mail":"andre@gmail.com",
        "x":4510,
        "y":14.058,
        "active":False
    },
    {
        "nombre":"Juan",
        "nacimiento": date(2000,9,7),
        "mail":"juan@gmail.com",
        "x":3000,
        "y":1.2,
        "active":True
    }
]

df1 = spark.createDataFrame(data)


In [10]:
df1.show()

+------+------+----------+-------+----+------+
|active|género|nacimiento| nombre|   x|     y|
+------+------+----------+-------+----+------+
|  true|     M|1987-10-07|Marcelo|5689| 15.26|
| false|     F|1990-08-19| Andrea|4510|14.058|
|  true|     M|2000-09-07|   Juan|3000|   1.2|
+------+------+----------+-------+----+------+



In [11]:
df1.limit(1).show()

+------+------+----------+-------+----+-----+
|active|género|nacimiento| nombre|   x|    y|
+------+------+----------+-------+----+-----+
|  true|     M|1987-10-07|Marcelo|5689|15.26|
+------+------+----------+-------+----+-----+



In [19]:
from pyspark.sql import Row

df2 = spark.createDataFrame([
    Row(nombre="Marcelo", nacimiento=date(1987,10,7), mail="marcelo.medel.v@gmail.com", x=2356, y=23.56, active=True),
    Row(nombre="Andrea", nacimiento=date(1990,8,19), mail="andrea@gmail.com", x=6352, y=1.56, active=False),
    Row(nombre="Juan", nacimiento=date(2000,9,19), mail="juan@gmail.com", x=5555, y=222.56, active=False)
])

df2.show()

+-------+----------+--------------------+----+------+------+
| nombre|nacimiento|                mail|   x|     y|active|
+-------+----------+--------------------+----+------+------+
|Marcelo|1987-10-07|marcelo.medel.v@g...|2356| 23.56|  true|
| Andrea|1990-08-19|    andrea@gmail.com|6352|  1.56| false|
|   Juan|2000-09-19|      juan@gmail.com|5555|222.56| false|
+-------+----------+--------------------+----+------+------+



In [20]:
df1.printSchema()
df2.printSchema()

root
 |-- active: boolean (nullable = true)
 |-- género: string (nullable = true)
 |-- nacimiento: date (nullable = true)
 |-- nombre: string (nullable = true)
 |-- x: long (nullable = true)
 |-- y: double (nullable = true)

root
 |-- nombre: string (nullable = true)
 |-- nacimiento: date (nullable = true)
 |-- mail: string (nullable = true)
 |-- x: long (nullable = true)
 |-- y: double (nullable = true)
 |-- active: boolean (nullable = true)



### Data Types

In [27]:
from pyspark.sql.types import (
    StringType, #texto
    IntegerType, # entero
    DateType, # datetime.date
    FloatType, # 2.33
    DecimalType, # DecimalType(10,2)
    BooleanType, #True or False
    ArrayType, # Lista, tupla...
    StructType, # colección de StructField
    StructField, # un campo estructurado
    MapType,
    LongType,
    DoubleType
)

cols = [
    #StructField("nombre", StringType(), False),
    StructField("nombre_completo", StructType([
        StructField("nombre", StringType(), False),
        StructField("apellido", StringType(), False)
    ]) ),
    StructField("nacimiento", DateType(), True),
    StructField("mail", StringType(), True),
    StructField("x", IntegerType(), False),
    StructField("y", FloatType(), False),
    StructField("active", BooleanType(), False),
]

schema = StructType(cols)

df3 = spark.createDataFrame([], schema)

df3.show()
df3.printSchema()

+---------------+----------+----+---+---+------+
|nombre_completo|nacimiento|mail|  x|  y|active|
+---------------+----------+----+---+---+------+
+---------------+----------+----+---+---+------+

root
 |-- nombre_completo: struct (nullable = true)
 |    |-- nombre: string (nullable = false)
 |    |-- apellido: string (nullable = false)
 |-- nacimiento: date (nullable = true)
 |-- mail: string (nullable = true)
 |-- x: integer (nullable = false)
 |-- y: float (nullable = false)
 |-- active: boolean (nullable = false)



In [32]:
data = [
    {
        "nombre_completo":["Marcelo","Medel"],
        "nacimiento": date(1987,10,7),
        "mail":"marcelo@gmail.com",
        "x":5689,
        "y":15.26,
        "active":True
    },
    {
        "nombre_completo":["Andrea","Vergara"],
        "nacimiento": date(1990,8,19),
        "mail":"andre@gmail.com",
        "x":4510,
        "y":14.058,
        "active":False
    },
    {
        "nombre_completo":["Juan","Bautista"],
        "nacimiento": date(2000,9,7),
        "mail":"juan@gmail.com",
        "x":3000,
        "y":1.2,
        "active":True
    }
]

df = spark.createDataFrame(data,schema)
df.show()
df.printSchema()

+-----------------+----------+-----------------+----+------+------+
|  nombre_completo|nacimiento|             mail|   x|     y|active|
+-----------------+----------+-----------------+----+------+------+
| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com|5689| 15.26|  true|
|{Andrea, Vergara}|1990-08-19|  andre@gmail.com|4510|14.058| false|
| {Juan, Bautista}|2000-09-07|   juan@gmail.com|3000|   1.2|  true|
+-----------------+----------+-----------------+----+------+------+

root
 |-- nombre_completo: struct (nullable = true)
 |    |-- nombre: string (nullable = false)
 |    |-- apellido: string (nullable = false)
 |-- nacimiento: date (nullable = true)
 |-- mail: string (nullable = true)
 |-- x: integer (nullable = false)
 |-- y: float (nullable = false)
 |-- active: boolean (nullable = false)



## Operaciones básicas sobre un DataFrame

### Select, Columns y Expressions

***Select*** permite seleccionar un set de columnas y también puede ser usado para renombrar o aplicar *expresiones* a las columnas.

***Columns*** en Spark hace referencia a cualquier columna conocida en algun RDBMS, planilla de excel, pandas dataframes, etc. Estas columnas pueden ser seleccionadas, manipuladas o removidas de un DataFrame, por lo que estas operaciones son representadas como ***expressions***

-  `col` y `column` son utilizadas para referencias a columnas en un DataFrame
- `expr` es utilizado para crear expresiones mas complejas, recibe un string con SQL para ejecutar la operación.


### Literals, withColumn, withColumnRenamed y alias

### Filter, where, drop y cast

### Pandas DataFrame to Spark DataFrame

![spark-pandas](./files/spark_pandas.png)